In [1]:
import os, sys
import time
import numpy as np
import pandas as pd
from scipy import stats as st
import itertools
from collections import defaultdict

# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline

import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator
from ipywidgets import interact, widgets

# Inits

In [93]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb')
%run $helpers_file

## uncomment the following lines if you want to run the various strategies in budget_strategies
# implemented_strategies = os.path.join(base_dir, 'budget_strategies.ipynb')
# %run $implemented_strategies
%store -r random_cfs
%store -r brute_force_cfs

%store -r bfs_yloss_cfs
# %store -r dfs_yloss_cfs
# %store -r bestfs_yloss_cfs

%store -r bidirectional_cfs
%store -r combo_cfs

In [110]:
from collections import Counter


# cnt = dict.fromkeys(most_sim_cfs.keys())
# no_target_achieved_cases = dict.fromkeys(most_sim_cfs.keys())
cnt = defaultdict(dict)
no_target_achieved_cases = defaultdict(list)

for i in range(1):
    cnt, no_target_achieved_cases = convert_res_to_lists(random_cfs[i], cnt, no_target_achieved_cases, f'random_{i}')
    cnt, no_target_achieved_cases = convert_res_to_lists(brute_force_cfs[i], cnt, no_target_achieved_cases, f'brute_force_{i}')

    cnt, no_target_achieved_cases = convert_res_to_lists(bfs_yloss_cfs[i], cnt, no_target_achieved_cases, f'bfs_{i}')
    cnt, no_target_achieved_cases = convert_res_to_lists(bfs_yloss_cfs[i + 1], cnt, no_target_achieved_cases, f'bfs_interrupted_{i}')
#     cnt, no_target_achieved_cases = convert_res_to_lists(dfs_yloss_cfs[i], cnt, no_target_achieved_cases, f'dfs_{i}')
#     cnt, no_target_achieved_cases = convert_res_to_lists(bestfs_yloss_cfs[i], cnt, no_target_achieved_cases, f'bestfs_{i}')

    cnt, no_target_achieved_cases = convert_res_to_lists(bidirectional_cfs[i], cnt, no_target_achieved_cases, f'bidirectional_001_{i}')
    cnt, no_target_achieved_cases = convert_res_to_lists(bidirectional_cfs[i + 1], cnt, no_target_achieved_cases, f'bidirectional_5_{i}')
    cnt, no_target_achieved_cases = convert_res_to_lists(bidirectional_cfs[i + 2], cnt, no_target_achieved_cases, f'bidirectional_999_{i}')

    cnt, no_target_achieved_cases = convert_res_to_lists(combo_cfs[i], cnt, no_target_achieved_cases, f'combo_001_{i}')
    cnt, no_target_achieved_cases = convert_res_to_lists(combo_cfs[i + 1], cnt, no_target_achieved_cases, f'combo_5_{i}')
    cnt, no_target_achieved_cases = convert_res_to_lists(combo_cfs[i + 2], cnt, no_target_achieved_cases, f'combo_999_{i}')

In [111]:
# sns.set_theme(style="ticks")
strategies = [('init', 2), ('best', 3)]
# pos = [2]
# strategies = ['init']
custom_range = [9, 29]
# target_pos = [3]
target_pos = list(cnt.keys())
# cnames = ['user_id', 'method'] + [f'card_{str(idx + 1)}' for idx in range(custom_range[1] - custom_range[0])]
cnames = ['user_id', 'method', 'pos', 'budget', 'init_budget', 'best_budget', 'cardinality', 'cfs_orig', 'cfs']

# Initialize the figure with a logarithmic x axis
# f, ax = plt.subplots(3, 1, figsize=(20, 20))
rows, cols = 5, 1
# fig, axs = plt.subplots(rows, cols, figsize=(25, 20))

tmp_dfs = []
for col in range(cols):
    for pos, row in zip(target_pos, range(rows)):

        tmp_dfs.append(pd.DataFrame.from_records(
            list(itertools.chain(*(
                zip(v[6], itertools.repeat(k), itertools.repeat(pos), v[i], v[2], v[3], itertools.repeat(i - custom_range[0] + 1), v[7], v[8])
                for k, v in cnt[pos].items() if f'_{col}' in k
                for i in range(custom_range[0], custom_range[1])
            ))), columns=cnames
        ))

df = pd.concat(tmp_dfs, ignore_index=True)

## Store dataframe

In [112]:
# df.to_parquet('../output/cfs_df.parquet')

In [119]:
# methods_supported = ['Brute_Force', 'DFS', 'BFS', 'BestFS', 'Random', 'Combo'] + [f'BiDirectional_{m}' for m in ['0', '01', '05']]
# methods_supported = ['Brute_Force', 'Combo', 'BFS_interrupted', 'Random'] + [f'BiDirectional_{m}' for m in ['0']]
# methods_supported = ['Brute_Force'] + [f'BiDirectional_{m}' for m in ['softmax', 'rescale', 'kth_norm']]
# methods_supported = ['Brute_Force', 'BFS', 'BFS_interrupted']
# methods_supported = ['Brute_Force', 'Combo', 'BFS_interrupted', 'Combo_tt']
# methods_supported = ['Brute_Force', 'Combo_t', 'Combo_t1', 'Combo_t2', 'Combo_t3', 'Combo_t4']
# methods_supported = ['Brute_Force', 'Random', 'BFS'] + [f'BiDirectional_{m}' for m in ['001', '5', '999']] + [f'Combo_{m}' for m in ['001', '5', '999']]
methods_supported = ['Brute_Force', 'BFS'] + [f'BiDirectional_{m}' for m in ['999']] + [f'Combo_{m}' for m in ['999']]

easy = [0, 3]
medium = [3, 5]
difficult = [5, 21]
# max_cardinality = 20
groupby_method = 'brute_force'


df.loc[(df['init_budget'] != df['budget']), ['init_budget']] = 0
df.loc[(df['best_budget'] != df['budget']), ['best_budget']] = 0


for m in methods_supported:
    for p in df.groupby('pos').groups.keys():
#     for p in [3]:
        restrictions = (df['method'] == f'{m.lower()}_0') & (df['pos'] == p)

#         g = df[restrictions].groupby(['user_id'])['budget']
#         idx = df.loc[restrictions & (g.transform(sum)==0)].drop_duplicates(['user_id'], keep='last').index
        idx = df.loc[restrictions & (df['cfs_orig'] == df['cfs'])].drop_duplicates(['user_id'], keep='last').index
        df.loc[idx, 'max_card'] = df[restrictions]['cardinality'].agg(max)
        df.loc[idx, ['budget', 'init_budget', 'best_budget']] = np.random.randint(100, df[restrictions]['budget'].agg(max))

        # update restrictions
        restrictions = (df['method'] == f'{m.lower()}_0') & (df['budget'] > 0) & (df['pos'] == p)

        g = df[restrictions].groupby(['budget', 'cardinality'])['budget']
        df.loc[restrictions, 'times'] = g.transform('count')

        g = df[restrictions].groupby(['user_id'])['cardinality']
        df.loc[restrictions, 'max_card'] = g.transform('max')

        if groupby_method in m.lower():
            df.loc[restrictions & (df['max_card'] >= difficult[0]), 'category'] = 'hard'
            df.loc[restrictions & (df['max_card'] < medium[1]) & (df['max_card'] >= medium[0]), 'category'] = 'medium'
            df.loc[restrictions & (df['max_card'] < easy[1]), 'category'] = 'easy'
        else:
            for k in ['easy', 'medium', 'hard']:
                user_ids = df[
                    (df['budget'] > 0) & (df['pos'] == p) & (df['category'] == k) &
                    (groupby_method in ' '.join(list(df.groupby('method')['method'].groups.keys())))
                ]['user_id'].to_numpy()
                df.loc[restrictions & df['user_id'].isin(user_ids), 'category'] = k

# df['times'] = df['times'].astype('int')

# Plots

In [120]:
layout = widgets.Layout(width='auto')
style = {'description_width': 'initial'}

budget = widgets.IntSlider(
    value=10000.0,
    min=50.0,
    max=df['budget'].agg('max') + 50.0,
    step=50,
    description='Available budget:',
    continuous_update=False,
    style=style,
#     layout=layout
)

textbox = widgets.Dropdown(
    description='Group of users to preview: ',
    value='all',
    options=['easy', 'medium', 'hard', 'all'],
    style=style,
)

options = [(i, j) for i, j in zip(methods_supported, range(len(methods_supported)))]
# strategies = widgets.SelectMultiple(
#     options=options,
#     value=[0],
#     rows=len(options),
#     description='Strategies',
#     disabled=False,
# )

valid_solutions = widgets.Dropdown(
    description='Skyline: ',
    value='all',
    options=['init', 'best', 'all']
)

items = widgets.Dropdown(
    description='Items: ',
    value='all',
    options=[5, 10, 20, 200, 'all']
)

t_pos = widgets.Dropdown(
    description='Target position: ',
    value=3,
    options=df.groupby(['pos'])['pos'].groups.keys(),
    style=style,
)

restrictions = (df['budget'] > 0) & (df['pos'] == t_pos.value)
raw_symbols = SymbolValidator().values
data = []
for i, m in enumerate(options):
    data.append(go.Scatter(
        y=df[restrictions & (df['method'] == f'{m[0].lower()}_0')]['budget'],
        x=df[restrictions & (df['method'] == f'{m[0].lower()}_0')]['cardinality'],
        mode='markers',
        marker_symbol=raw_symbols[12*i + 2],
    #     opacity=0.5,
    #     marker_size=df['counts'],
        marker=dict(
            size=[10] * len(df[restrictions & (df['method'] == f'{m[0].lower()}_0')]),
    #         color=df[restrictions]['times'], #set color equal to a variable
            colorscale='Viridis', # one of plotly colorscales
    #         showscale=True,
            opacity=0.5,
    #         line=dict(
    #             color='DarkSlateGrey',
    #             width=1
    #         ),
    #         color='rgb(255, 178, 102)',
        ),
    #     text=[
    #         f'budget: {round(yval,2)}<br>uid: {uval}<br>card: {card}'
    #         for yval, uval, card in zip(
    #             df[restrictions]['budget'], df[restrictions]['user_id'], df[restrictions]['cardinality']
    #         )
    #     ],
        meta=df[restrictions & (df['method'] == f'{m[0].lower()}_0')]['user_id'],
        hoverlabel=dict(namelength=0),
        hovertemplate='uid: %{meta}<br>budget: %{y} <br>cardinality: %{x}',
        name=m[0],
#         visible=True if m[1] in strategies.value else False
    ))

layout = dict(
    title='Budget-Quality Analysis',
    xaxis=dict(title='Cardinality'),
    yaxis=dict(title='Budget'),
    showlegend=True,
    hovermode='closest',
#     width=1000,
    height=600,
)

fw1 = go.FigureWidget(data=data, layout=layout)
# fig_subplots = widgets.VBox([fw1, fw2])

In [121]:
from plotly.callbacks import Points


points = Points()


def selection_fn(trace, points, selector):
    s = list(trace.marker.size)
#     l = list(trace.marker.line.color)
    inds = points.point_inds

    # Do something
    for i in inds:
#         to_update = (trace.meta == trace.meta[i])
        to_update = (fw1.data[0].meta == trace.meta[i])
        for ii in itertools.compress(range(len(to_update)), to_update):
            s[ii] = 13

        with fw1.batch_update():
            fw1.data[0].marker.size = s
#             for d in fw1.data:
#                 d.marker.size = s


def deselect_fn(trace, points):
    inds = points.point_inds

    # Do something
    print(inds)


for fig in fw1.data:
    fig.on_click(selection_fn)
    fig.on_deselect(deselect_fn)


def response(change):
#     colorsIdx = ["black", "red", "cyan", "magenta", "blue", "green3", "white", "yellow", ]
    budget_type = 'budget' if valid_solutions.value == 'all' else f'{valid_solutions.value}_budget'
    filter_list = None
#     if validate():
#         if use_date.value:
    if textbox.value == 'all':
        filter_list = [i for i in df[budget_type] <= budget.value]
    else:
        filter_list = [i and j for i, j in zip(
            df[budget_type] <= budget.value,
            df['category'] == textbox.value
        )]
    filter_list &= (df[budget_type] > 0) & (df['pos'] == t_pos.value)

    if items.value != 'all':
        user_ids = np.random.choice(pd.unique(df[filter_list]['user_id']), items.value)
        filter_list &= (df['user_id'].isin(user_ids))

#     g = temp_df.groupby(['budget','cardinality'])[budget_type]
#     temp_df.loc[temp_df.index, 'times'] = g.transform('count')
#     c = temp_df['times']
#     c = temp_df['method'].map({m: colorsIdx[i] for i, m in enumerate(methods)})
    with fw1.batch_update():
        for i in range(len(fw1.data)):
            custom_filter = (df['method'] == f'{methods_supported[i].lower()}_0')
            temp_df = df[filter_list & custom_filter]

            y = temp_df[budget_type]
            x = temp_df['cardinality']

#             if i in strategies.value:
#                 fw1.data[i].visible = True

            fw1.data[i].y = y
            fw1.data[i].x = x
            fw1.data[i].meta = temp_df['user_id']
    #         fw1.data[0].marker.color = c


textbox.observe(response, names="value")
budget.observe(response, names="value")
# strategies.observe(response, names="value")
valid_solutions.observe(response, names="value")
items.observe(response, names='value')
t_pos.observe(response, names='value')

container = widgets.HBox([budget, textbox, valid_solutions])
container2 = widgets.HBox([items, t_pos])
widgets.VBox([container, container2, fw1])

# Inits

In [100]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb')
%run $helpers_file

## uncomment the following lines if you want to run the various strategies in budget_strategies
# implemented_strategies = os.path.join(base_dir, 'budget_strategies.ipynb')
# %run $implemented_strategies
%store -r amazon_random_cfs
%store -r amazon_brute_force_cfs

%store -r amazon_bfs_yloss_cfs
# %store -r dfs_yloss_cfs
# %store -r bestfs_yloss_cfs

# %store -r amazon_bidirectional_cfs
# %store -r amazon_combo_cfs

In [103]:
from collections import Counter


# cnt = dict.fromkeys(most_sim_cfs.keys())
# no_target_achieved_cases = dict.fromkeys(most_sim_cfs.keys())
cnt = defaultdict(dict)
no_target_achieved_cases = defaultdict(list)

for i in range(1):
    cnt, no_target_achieved_cases = convert_res_to_lists(amazon_random_cfs[i], cnt, no_target_achieved_cases, f'random_{i}')
    cnt, no_target_achieved_cases = convert_res_to_lists(amazon_brute_force_cfs[i], cnt, no_target_achieved_cases, f'brute_force_{i}')

    cnt, no_target_achieved_cases = convert_res_to_lists(amazon_bfs_yloss_cfs[i], cnt, no_target_achieved_cases, f'bfs_{i}')
#     cnt, no_target_achieved_cases = convert_res_to_lists(amazon_bfs_yloss_cfs[i + 1], cnt, no_target_achieved_cases, f'bfs_interrupted_{i}')
#     cnt, no_target_achieved_cases = convert_res_to_lists(dfs_yloss_cfs[i], cnt, no_target_achieved_cases, f'dfs_{i}')
#     cnt, no_target_achieved_cases = convert_res_to_lists(bestfs_yloss_cfs[i], cnt, no_target_achieved_cases, f'bestfs_{i}')

#     cnt, no_target_achieved_cases = convert_res_to_lists(amazon_bidirectional_cfs[i], cnt, no_target_achieved_cases, f'bidirectional_001_{i}')
#     cnt, no_target_achieved_cases = convert_res_to_lists(amazon_bidirectional_cfs[i + 1], cnt, no_target_achieved_cases, f'bidirectional_5_{i}')
#     cnt, no_target_achieved_cases = convert_res_to_lists(amazon_bidirectional_cfs[i + 2], cnt, no_target_achieved_cases, f'bidirectional_999_{i}')

#     cnt, no_target_achieved_cases = convert_res_to_lists(amazon_combo_cfs[i], cnt, no_target_achieved_cases, f'combo_001_{i}')
#     cnt, no_target_achieved_cases = convert_res_to_lists(amazon_combo_cfs[i + 1], cnt, no_target_achieved_cases, f'combo_5_{i}')
#     cnt, no_target_achieved_cases = convert_res_to_lists(amazon_combo_cfs[i + 2], cnt, no_target_achieved_cases, f'combo_999_{i}')

In [104]:
# sns.set_theme(style="ticks")
strategies = [('init', 2), ('best', 3)]
# pos = [2]
# strategies = ['init']
custom_range = [9, 29]
# target_pos = [3]
target_pos = list(cnt.keys())
# cnames = ['user_id', 'method'] + [f'card_{str(idx + 1)}' for idx in range(custom_range[1] - custom_range[0])]
cnames = ['user_id', 'method', 'pos', 'budget', 'init_budget', 'best_budget', 'cardinality', 'cfs_orig', 'cfs']

# Initialize the figure with a logarithmic x axis
# f, ax = plt.subplots(3, 1, figsize=(20, 20))
rows, cols = 5, 1
# fig, axs = plt.subplots(rows, cols, figsize=(25, 20))

tmp_dfs = []
for col in range(cols):
    for pos, row in zip(target_pos, range(rows)):

        tmp_dfs.append(pd.DataFrame.from_records(
            list(itertools.chain(*(
                zip(v[6], itertools.repeat(k), itertools.repeat(pos), v[i], v[2], v[3], itertools.repeat(i - custom_range[0] + 1), v[7], v[8])
                for k, v in cnt[pos].items() if f'_{col}' in k
                for i in range(custom_range[0], custom_range[1])
            ))), columns=cnames
        ))

df = pd.concat(tmp_dfs, ignore_index=True)

## Store dataframe

In [105]:
# df.to_parquet('../output/cfs_df.parquet')

In [107]:
# methods_supported = ['Brute_Force', 'DFS', 'BFS', 'BestFS', 'Random', 'Combo'] + [f'BiDirectional_{m}' for m in ['0', '01', '05']]
# methods_supported = ['Brute_Force', 'Combo', 'BFS_interrupted', 'Random'] + [f'BiDirectional_{m}' for m in ['0']]
# methods_supported = ['Brute_Force'] + [f'BiDirectional_{m}' for m in ['softmax', 'rescale', 'kth_norm']]
# methods_supported = ['Brute_Force', 'BFS', 'BFS_interrupted']
# methods_supported = ['Brute_Force', 'Combo', 'BFS_interrupted', 'Combo_tt']
# methods_supported = ['Brute_Force', 'Combo_t', 'Combo_t1', 'Combo_t2', 'Combo_t3', 'Combo_t4']
methods_supported = ['Brute_Force', 'Random', 'BFS']  # + [f'BiDirectional_{m}' for m in ['001', '5', '999']] + [f'Combo_{m}' for m in ['001', '5', '999']]

easy = [0, 3]
medium = [3, 5]
difficult = [5, 21]
# max_cardinality = 20
groupby_method = 'brute_force'


df.loc[(df['init_budget'] != df['budget']), ['init_budget']] = 0
df.loc[(df['best_budget'] != df['budget']), ['best_budget']] = 0


for m in methods_supported:
#     for p in df.groupby('pos').groups.keys():
    for p in [3]:
        restrictions = (df['method'] == f'{m.lower()}_0') & (df['pos'] == p)

#         g = df[restrictions].groupby(['user_id'])['budget']
#         idx = df.loc[restrictions & (g.transform(sum)==0)].drop_duplicates(['user_id'], keep='last').index
        idx = df.loc[restrictions & (df['cfs_orig'] == df['cfs'])].drop_duplicates(['user_id'], keep='last').index
        df.loc[idx, 'max_card'] = df[restrictions]['cardinality'].agg(max)
        df.loc[idx, ['budget', 'init_budget', 'best_budget']] = np.random.randint(100, df[restrictions]['budget'].agg(max))

        # update restrictions
        restrictions = (df['method'] == f'{m.lower()}_0') & (df['budget'] > 0) & (df['pos'] == p)

        g = df[restrictions].groupby(['budget', 'cardinality'])['budget']
        df.loc[restrictions, 'times'] = g.transform('count')

        g = df[restrictions].groupby(['user_id'])['cardinality']
        df.loc[restrictions, 'max_card'] = g.transform('max')

        if groupby_method in m.lower():
            df.loc[restrictions & (df['max_card'] >= difficult[0]), 'category'] = 'hard'
            df.loc[restrictions & (df['max_card'] < medium[1]) & (df['max_card'] >= medium[0]), 'category'] = 'medium'
            df.loc[restrictions & (df['max_card'] < easy[1]), 'category'] = 'easy'
        else:
            for k in ['easy', 'medium', 'hard']:
                user_ids = df[
                    (df['budget'] > 0) & (df['pos'] == p) & (df['category'] == k) &
                    (groupby_method in ' '.join(list(df.groupby('method')['method'].groups.keys())))
                ]['user_id'].to_numpy()
                df.loc[restrictions & df['user_id'].isin(user_ids), 'category'] = k

# df['times'] = df['times'].astype('int')

# Plots

In [108]:
layout = widgets.Layout(width='auto')
style = {'description_width': 'initial'}

budget = widgets.IntSlider(
    value=10000.0,
    min=50.0,
    max=df['budget'].agg('max') + 50.0,
    step=50,
    description='Available budget:',
    continuous_update=False,
    style=style,
#     layout=layout
)

textbox = widgets.Dropdown(
    description='Group of users to preview: ',
    value='all',
    options=['easy', 'medium', 'hard', 'all'],
    style=style,
)

options = [(i, j) for i, j in zip(methods_supported, range(len(methods_supported)))]
# strategies = widgets.SelectMultiple(
#     options=options,
#     value=[0],
#     rows=len(options),
#     description='Strategies',
#     disabled=False,
# )

valid_solutions = widgets.Dropdown(
    description='Skyline: ',
    value='all',
    options=['init', 'best', 'all']
)

items = widgets.Dropdown(
    description='Items: ',
    value='all',
    options=[5, 10, 20, 200, 'all']
)

t_pos = widgets.Dropdown(
    description='Target position: ',
    value=3,
    options=df.groupby(['pos'])['pos'].groups.keys(),
    style=style,
)

restrictions = (df['budget'] > 0) & (df['pos'] == t_pos.value)
raw_symbols = SymbolValidator().values
data = []
for i, m in enumerate(options):
    data.append(go.Scatter(
        y=df[restrictions & (df['method'] == f'{m[0].lower()}_0')]['budget'],
        x=df[restrictions & (df['method'] == f'{m[0].lower()}_0')]['cardinality'],
        mode='markers',
        marker_symbol=raw_symbols[12*i + 2],
    #     opacity=0.5,
    #     marker_size=df['counts'],
        marker=dict(
            size=[10] * len(df[restrictions & (df['method'] == f'{m[0].lower()}_0')]),
    #         color=df[restrictions]['times'], #set color equal to a variable
            colorscale='Viridis', # one of plotly colorscales
    #         showscale=True,
            opacity=0.5,
    #         line=dict(
    #             color='DarkSlateGrey',
    #             width=1
    #         ),
    #         color='rgb(255, 178, 102)',
        ),
    #     text=[
    #         f'budget: {round(yval,2)}<br>uid: {uval}<br>card: {card}'
    #         for yval, uval, card in zip(
    #             df[restrictions]['budget'], df[restrictions]['user_id'], df[restrictions]['cardinality']
    #         )
    #     ],
        meta=df[restrictions & (df['method'] == f'{m[0].lower()}_0')]['user_id'],
        hoverlabel=dict(namelength=0),
        hovertemplate='uid: %{meta}<br>budget: %{y} <br>cardinality: %{x}',
        name=m[0],
#         visible=True if m[1] in strategies.value else False
    ))

layout = dict(
    title='Budget-Quality Analysis',
    xaxis=dict(title='Cardinality'),
    yaxis=dict(title='Budget'),
    showlegend=True,
    hovermode='closest',
#     width=1000,
    height=600,
)

fw1 = go.FigureWidget(data=data, layout=layout)
# fig_subplots = widgets.VBox([fw1, fw2])

In [109]:
from plotly.callbacks import Points


points = Points()


def selection_fn(trace, points, selector):
    s = list(trace.marker.size)
#     l = list(trace.marker.line.color)
    inds = points.point_inds

    # Do something
    for i in inds:
#         to_update = (trace.meta == trace.meta[i])
        to_update = (fw1.data[0].meta == trace.meta[i])
        for ii in itertools.compress(range(len(to_update)), to_update):
            s[ii] = 13

        with fw1.batch_update():
            fw1.data[0].marker.size = s
#             for d in fw1.data:
#                 d.marker.size = s


def deselect_fn(trace, points):
    inds = points.point_inds

    # Do something
    print(inds)


for fig in fw1.data:
    fig.on_click(selection_fn)
    fig.on_deselect(deselect_fn)


def response(change):
#     colorsIdx = ["black", "red", "cyan", "magenta", "blue", "green3", "white", "yellow", ]
    budget_type = 'budget' if valid_solutions.value == 'all' else f'{valid_solutions.value}_budget'
    filter_list = None
#     if validate():
#         if use_date.value:
    if textbox.value == 'all':
        filter_list = [i for i in df[budget_type] <= budget.value]
    else:
        filter_list = [i and j for i, j in zip(
            df[budget_type] <= budget.value,
            df['category'] == textbox.value
        )]
    filter_list &= (df[budget_type] > 0) & (df['pos'] == t_pos.value)

    if items.value != 'all':
        user_ids = np.random.choice(pd.unique(df[filter_list]['user_id']), items.value)
        filter_list &= (df['user_id'].isin(user_ids))

#     g = temp_df.groupby(['budget','cardinality'])[budget_type]
#     temp_df.loc[temp_df.index, 'times'] = g.transform('count')
#     c = temp_df['times']
#     c = temp_df['method'].map({m: colorsIdx[i] for i, m in enumerate(methods)})
    with fw1.batch_update():
        for i in range(len(fw1.data)):
            custom_filter = (df['method'] == f'{methods_supported[i].lower()}_0')
            temp_df = df[filter_list & custom_filter]

            y = temp_df[budget_type]
            x = temp_df['cardinality']

#             if i in strategies.value:
#                 fw1.data[i].visible = True

            fw1.data[i].y = y
            fw1.data[i].x = x
            fw1.data[i].meta = temp_df['user_id']
    #         fw1.data[0].marker.color = c


textbox.observe(response, names="value")
budget.observe(response, names="value")
# strategies.observe(response, names="value")
valid_solutions.observe(response, names="value")
items.observe(response, names='value')
t_pos.observe(response, names='value')

container = widgets.HBox([budget, textbox, valid_solutions])
container2 = widgets.HBox([items, t_pos])
widgets.VBox([container, container2, fw1])